In [2]:
import pandas as pd

rdi = 'https://raw.githubusercontent.com/alura-cursos/alura_classificacao_multilabel/master/dataset/stackoverflow_perguntas.csv'
dados = pd.read_csv(rdi)
dados.sample(10)

,Perguntas,Tags
1155,eu estou com este código: CODE CODE ...,html
4853,"Tipo, meu site tem vários posts e tal, gostari...",html
4033,Tenho a seguinte linha de código: CODE que ...,angular
2841,Estou tendo problemas em executar um código es...,node.js
4630,Estou fazendo uma consulta em uma API (SWAPI)....,angular
3203,"Estou desenvolvendo um site com node js, ele t...",node.js
1510,Gostaria de realizar uma simples animação que ...,jquery
3565,"Bom, eu tenho um sistema de comentário, onde e...",jquery html
968,"Tenho uma caixa de texto, uma input type text....",jquery html
2619,"O Problema Tenho a variável CODE , e preciso ...",jquery


In [3]:
dados.sample(10)

,Perguntas,Tags
3440,Estou precisando fazer NodeJS ler uma DLL para...,node.js
4187,Gostaria de saber como posso fazer para passar...,angular
5192,Tenho um formulário que tem um botão de marcar...,html
238,Como faço isso rodar ? CODE,html
3361,Tenho na pagina que lista os imóveis do meu si...,jquery
2667,Estou com umas tabs e quero deixar responsiva....,jquery html
4681,Estou tentando usar a função para alterar en...,html
3630,Tenho um CODE CODE O que eu preciso é que ...,html
4916,"Possuo 4 tables de informação, e gostaria de f...",html
5171,Tenho vários campos e outros em CODE (De ante...,jquery


In [4]:
dados.shape

(5408, 2)

In [5]:
print(len(dados.Tags.unique()))

37


In [6]:
dados.Tags.unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [7]:
label = list()
for tags in dados.Tags.unique():
    for tag in tags.split():
        if tag not in label:
            label.append(tag)

print(label)

['node.js', 'jquery', 'html', 'angular']


In [10]:
node_js = list()
for linha_tag in dados.Tags:
    if "node.js" in linha_tag:
        node_js.append(1)
    else:
        node_js.append(0)
dados['node.js'] = node_js
dados.head()

,Perguntas,Tags,node.js
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1
2,Como inverter a ordem com que o jQuery itera u...,jquery,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0
4,Como exibir os dados retornados do FireStore e...,html angular,0


In [11]:
lista = list(label)
for coluna in label:
    node_js = list()
    for linha_tag in dados.Tags:
        if coluna in linha_tag:
            node_js.append(1)
        else:
            node_js.append(0)
    dados[coluna] = node_js
dados.head()

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


In [27]:
lista_zip_tags = list(zip(dados[label[0]],dados[label[1]],dados[label[2]],dados[label[3]]))
dados['todas_tags'] = lista_zip_tags
dados.head()

In [30]:
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(dados.Perguntas, dados.todas_tags, test_size= 0.2,
                                                        random_state= 123)

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

vetorizar = TfidfVectorizer(max_features= 5000, max_df=0.85)
vetorizar.fit(dados['Perguntas'])
x_treino_tfidf = vetorizar.transform(x_treino)
x_teste_tfidf  = vetorizar.transform(x_teste )

print(x_treino_tfidf.shape)
print(x_teste_tfidf.shape)

(4326, 5000)
(1082, 5000)


In [41]:
import numpy as np

y_treino_array = np.asarray(list(y_treino))
y_teste_array = np.asarray(list(y_teste))
print(y_treino_array)
print(type(y_treino_array))

[[0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 ...
 [0 1 1 0]
 [0 0 1 0]
 [0 1 1 0]]
<class 'numpy.ndarray'>


In [53]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(estimator=regressao_logistica)
classificador_onevsrest.fit(x_treino_tfidf,y_treino_array)
resultado_onevsrest = classificador_onevsrest.score(x_teste_tfidf,y_teste_array)
print("Resultado %.2f%%" % (resultado_onevsrest*100))

Resultado 41.68%


In [57]:
from sklearn.metrics import hamming_loss

previsao_onevsrest = classificador_onevsrest.predict(x_teste_tfidf)
hamming_loss_onevsrest = hamming_loss(y_teste_array, previsao_onevsrest)
print("Resultado %.2f" % hamming_loss_onevsrest)

Resultado 0.19


In [59]:
dados.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


In [63]:
from skmultilearn.problem_transform import ClassifierChain

classifier_chain = ClassifierChain(regressao_logistica) 
classifier_chain.fit(x_treino_tfidf,y_treino_array)
resultado_classifier_chain = classifier_chain.score(x_teste_tfidf,y_teste_array)
print("Resultado %.2f%%" % (resultado_classifier_chain*100))

Resultado 49.82%


In [64]:
previsao_classifier_chain = classifier_chain.predict(x_teste_tfidf)
hamming_classifier_chain= hamming_loss(y_teste_array, previsao_classifier_chain)
print("Resultado %.2f" % hamming_classifier_chain)

Resultado 0.21


In [70]:
from skmultilearn.problem_transform import BinaryRelevance

classifier_br = BinaryRelevance(regressao_logistica) 
classifier_br.fit(x_treino_tfidf,y_treino_array)
resultado_classifier_br = classifier_br.score(x_teste_tfidf,y_teste_array)
print("Resultado %.2f%%" % (resultado_classifier_br*100))
previsao_classifier_br = classifier_br.predict(x_teste_tfidf)
hamming_classifier_br= hamming_loss(y_teste_array, previsao_classifier_br)
print("Resultado %.2f" % hamming_classifier_br)

Resultado 41.68%
Resultado 0.19


In [ ]:
from skmultilearn.adapt import MLkNN

classifier_mlknn = MLkNN()
classifier_mlknn.fit(x_treino_tfidf,y_treino_array)
resultado_classifier_mlknn = classifier_mlknn.score(x_teste_tfidf,y_teste_array)
print("Resultado %.2f%%" % (resultado_classifier_mlknn*100))
previsao_classifier_br = classifier_mlknn.predict(x_teste_tfidf)
hamming_classifier_mlknn= hamming_loss(y_teste_array, resultado_classifier_mlknn)
print("Resultado %.2f" % hamming_classifier_mlknn)